# qBraid-SDK Transpiler Demo

In [1]:
import numpy as np
from time import time

from qbraid import circuit_wrapper, SUPPORTED_PROGRAM_TYPES
from qbraid.interface import to_unitary, circuits_allclose, draw
from qbraid.interface._programs import bell_data

start_notebook = time()

/home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy/pyenv/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


The qbraid transpiler supports the following frontend modules / quantum program types:

In [ ]:
SUPPORTED_PROGRAM_TYPES

## Intro to `qbraid.circuit_wrapper` with Bell circuit example

Load test data containing a bell circuit function for each supported program type, along with the unitary matrix that each circuit represents / implements.

In [ ]:
bell_circuits, expected_u = bell_data()
print(expected_u.shape)

Randomly choose a source and target package

In [ ]:
pkgs = list(SUPPORTED_PROGRAM_TYPES.keys())
source = pkgs.pop(np.random.randint(len(pkgs)))
target = pkgs.pop(np.random.randint(len(pkgs) - 1))
print(f"{source} --> {target}")

Instantiate the circuit object in the source package and print its diagram

In [ ]:
source_bell = bell_circuits[source]()
print(f"{type(source_bell)}\n")
draw(source_bell)

Apply the qbraid circuit wrapper to the source circuit

In [ ]:
wrapped_circuit = circuit_wrapper(source_bell)
print(type(wrapped_circuit))

Use the transpile method to convert to the target circuit type and prints its diagram

In [ ]:
target_bell = wrapped_circuit.transpile(target)
print(f"{type(target_bell)}\n")
draw(target_bell)

Next we'll verify that the unitary representation of the target circuit is correct.

Here, we're using the sdk's `to_unitary` function, which interprets the type of the input circuit, calculates its unitary using methods from that circuits native module, and returns the resulting `numpy.ndarray`.

In [ ]:
target_u = to_unitary(target_bell)
np.allclose(target_u, expected_u)

## Now, a non-trivial example

In [ ]:
from qiskit import QuantumCircuit

In [ ]:
def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit

We'll start with a 4-qubit qiskit circuit that uses 15 unique gates

In [ ]:
qiskit_circuit = test_circuit()
print(f"{type(qiskit_circuit)}\n")
qiskit_circuit.draw()

Applying the circuit wrapper and transpiling to braket and cirq

In [ ]:
wrapped_circuit = circuit_wrapper(qiskit_circuit)

In [ ]:
braket_circuit = wrapped_circuit.transpile("braket")
print(f"{type(braket_circuit)}\n")
print(braket_circuit)

In [ ]:
cirq_circuit = wrapped_circuit.transpile("cirq")
print(f"{type(cirq_circuit)}\n")
print(cirq_circuit)

Qubit indexing varies between packages, so some circuit diagrams appear flipped, but the matrix representations are equivalent.

To verify, we'll use the sdk's `circuits_allclose` function, which applies the qbraid `to_unitary` function mentioned above to each of two input circuits, and passes the matricies to `np.allclose`, and returns the result.

In [ ]:
circuits_allclose(qiskit_circuit, braket_circuit) and circuits_allclose(
    braket_circuit, cirq_circuit
)

## Stress-testing against randomly generated circuits

As a final demo, we'll generate some even larger circuits, and do so randomly, to test the limits of the transpiler.

The qbraid-SDK has its own `random_circuit` function that takes in any supported package as an argument, but to show that there's no pre-processing or filtering going on behind the scenes, I'll use functions from cirq's testing module to generate circuits and to check equivalance after transpiling.

In [ ]:
import cirq

kwargs = {
    "qubits": np.random.randint(8, 11),
    "n_moments": np.random.randint(8, 11),
    "op_density": np.random.randint(80, 100) / 100,
    "random_state": np.random.randint(1, 11),
}

circuit_start = cirq.testing.random_circuit(**kwargs)
start_u = circuit_start.unitary()
print("num qubits:", len(circuit_start.all_qubits()))
print("num moments:", len(circuit_start))
print("op density:", kwargs["op_density"])
print(f"matrix dim: {start_u.shape}\n")
print(circuit_start)

Starting with this randomly generated circuit, we'll repeatedly apply the qbraid circuit wrapper and transpile from one supported package to the next until we arrive all the way back at a cirq circuit.

In [ ]:
braket_circuit = circuit_wrapper(circuit_start).transpile("braket")
print(type(braket_circuit))
# print(f"\n{braket_circuit}")

In [ ]:
pyquil_circuit = circuit_wrapper(braket_circuit).transpile("pyquil")
print(type(pyquil_circuit))
# print(f"\n{pyquil_circuit}")

In [ ]:
qiskit_circuit = circuit_wrapper(pyquil_circuit).transpile("qiskit")
print(type(qiskit_circuit))
# print(f"\n{qiskit_circuit}")

In [ ]:
pennylane_circuit = circuit_wrapper(qiskit_circuit).transpile("pennylane")
print(type(pennylane_circuit))
# print(f"\n{pennylane_circuit}")

In [ ]:
circuit_finish = circuit_wrapper(pennylane_circuit).transpile("cirq")
print(type(circuit_finish))
# print(f"\n{circuit_finish}")

Computing the final unitary and checking its shape

In [ ]:
finish_u = circuit_finish.unitary()
print(finish_u.shape)

In [ ]:
try:
    cirq.testing.assert_allclose_up_to_global_phase(start_u, finish_u, atol=1e-7)
    print("Test passed!")
except AssertionError:
    print("Test failed")

In [ ]:
runtime = round(time() - start_notebook, 2)
print(f"Notebook ran in {runtime}s")